In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

In [2]:
def distancia_euclidiana_km(lat1, lon1, lat2, lon2):
    # entrada: floats (graus). retorno: distância em km (aprox.)
    return np.sqrt((lat1 - lat2)**2 + (lon1 - lon2)**2) * 111.0

In [3]:
path_cimehgo = r"C:\Users\thiag\OneDrive\Área de Trabalho\TCC 2\Scripts\Arquivos Finais\CIMEHGO_FINAL.xlsx"
path_inmet   = r"C:\Users\thiag\OneDrive\Área de Trabalho\TCC 2\Scripts\Arquivos Finais\INMET_FINAL.xlsx"

path_locais  = r"C:\Users\thiag\OneDrive\Área de Trabalho\TCC 2\Scripts\Arquivos Finais\Locais_por_Data_GEOCODED_FINAL.xlsx"
path_relatos = r"C:\Users\thiag\OneDrive\Área de Trabalho\TCC 2\Scripts\Arquivos Finais\RELATOS_GEOCODED_FINAL.xlsx"
path_bombe   = r"C:\Users\thiag\OneDrive\Área de Trabalho\TCC 2\Scripts\Arquivos Finais\BOMBEIROS_GEOCODED_FINAL.xlsx"

out_folder_bi = r"C:\Users\thiag\OneDrive\Área de Trabalho\TCC 2\Scripts\Arquivos para BI"
os.makedirs(out_folder_bi, exist_ok=True)

# Estações

In [4]:
cimehgo = pd.read_excel(path_cimehgo)
inmet   = pd.read_excel(path_inmet)

In [5]:
cimehgo_p = cimehgo.rename(columns={
    "ESTAÇÃO": "Estacao",
    "DATA/HORA(LOCAL)": "Data",
    "CHUVA(MM)": "Chuva_mm"
})
inmet_p = inmet.rename(columns={
    "ESTACAO": "Estacao",
    "Data": "Data",
    "Chuva(mm)": "Chuva_mm",                   
})

In [6]:
inmet_p["Chuva_mm"]   = pd.to_numeric(inmet_p["Chuva_mm"], errors="coerce").fillna(0)
cimehgo_p["Chuva_mm"] = pd.to_numeric(cimehgo_p["Chuva_mm"], errors="coerce").fillna(0)


if "Hora" in inmet_p.columns:
    inmet_p["Hora"] = pd.to_datetime(inmet_p["Hora"], format="%H:%M", errors="coerce").dt.time
    inmet_p["Data"] = pd.to_datetime(inmet_p["Data"], errors="coerce")
    inmet_p["Data"] = inmet_p.apply(lambda row: pd.Timestamp.combine(row["Data"], row["Hora"]) if pd.notna(row["Data"]) and pd.notna(row["Hora"]) else row["Data"], axis=1)
else:
    inmet_p["Data"] = pd.to_datetime(inmet_p["Data"], errors="coerce")

cimehgo_p["Data"] = pd.to_datetime(cimehgo_p["Data"], errors="coerce")

In [7]:
cimehgo_sel = cimehgo_p[["Estacao", "Data", "Chuva_mm", "LATITUDE", "LONGITUDE"]]
inmet_sel   = inmet_p[["Estacao", "Data", "Chuva_mm", "LATITUDE", "LONGITUDE"]]

df_chuva_unificada = pd.concat([cimehgo_sel, inmet_sel], ignore_index=True)
df_chuva_unificada["data_dia"] = df_chuva_unificada["Data"].dt.date

In [8]:
df_chuva_unificada.to_csv(os.path.join(out_folder_bi, "DADOS_UNIFICADOS_ESTACOES.csv"), index=False, encoding="latin1")

# Ocorrências

In [9]:
lp = pd.read_excel(path_locais)
rp = pd.read_excel(path_relatos)
bp = pd.read_excel(path_bombe)

bp = bp.rename(columns={"ENDEREÇO": "LOCAL"})

In [10]:
locais   = lp[["DATA", "LOCAL", "Estação atribuída", "LATITUDE", "LONGITUDE"]].copy()
relatos  = rp[["DATA", "LOCAL", "Estação atribuída", "LATITUDE", "LONGITUDE"]].copy()
bombeiros= bp[["DATA", "LOCAL", "Estação atribuída", "LATITUDE", "LONGITUDE"]].copy()

locais["Fonte"] = "Noticias"
relatos["Fonte"] = "Relatos"
bombeiros["Fonte"] = "Bombeiros"

In [11]:
def parse_date(x):
    if pd.isna(x):
        return pd.NaT
    s = str(x).strip()
    if "," in s:
        partes = s.split(",", 1)
        if len(partes) == 2:
            s = partes[1].strip()
    try:
        return pd.to_datetime(s, errors="raise", infer_datetime_format=True)
    except:
        pass
    try:
        return datetime.strptime(s, "%B %d %Y")
    except:
        pass
    try:
        return datetime.strptime(s, "%B %d, %Y")
    except:
        pass
    return pd.NaT

locais["DATA"] = locais["DATA"].apply(parse_date)
relatos["DATA"] = relatos["DATA"].apply(parse_date)
bombeiros["DATA"] = bombeiros["DATA"].apply(parse_date)

C:\Users\thiag\AppData\Local\Temp\ipykernel_22412\1656189541.py:10: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(s, errors="raise", infer_datetime_format=True)
C:\Users\thiag\AppData\Local\Temp\ipykernel_22412\1656189541.py:10: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(s, errors="raise", infer_datetime_format=True)
C:\Users\thiag\AppData\Local\Temp\ipykernel_22412\1656189541.py:10: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it 

In [12]:
df_ocorrencias = pd.concat([locais, relatos, bombeiros], ignore_index=True)

In [13]:
df_ocorrencias["DATA"] = pd.to_datetime(df_ocorrencias["DATA"], errors="coerce")
df_ocorrencias["data_dia"] = df_ocorrencias["DATA"].dt.date

# Analise

In [14]:
stations_meta = df_chuva_unificada[["Estacao", "LATITUDE", "LONGITUDE"]].drop_duplicates().dropna(subset=["LATITUDE", "LONGITUDE"]).reset_index(drop=True)

In [15]:
stations_lat = stations_meta["LATITUDE"].to_numpy(dtype=float)
stations_lon = stations_meta["LONGITUDE"].to_numpy(dtype=float)
stations_names = stations_meta["Estacao"].to_numpy(dtype=object)

In [16]:
RAIOS = [1.5, 5, 10, 20]

# -----------------------
# Loop principal: para cada raio, reatribui ocorrencias -> junta chuva -> salva outputs
# -----------------------
for raio_km in RAIOS:
    print(f"\n=== Processando raio {raio_km} km ===")
    # copia do df de ocorrências para manipular
    occ = df_ocorrencias.copy().reset_index(drop=True)

    # prepara colunas de atribuição
    occ["Estacao_reatribuida"] = None
    occ["Dist_km_reatribuida"] = None

    # itera sobre ocorrências (vectorização parcial)
    # extrai arrays de lat/lon ocorrência
    occ_lat = occ["LATITUDE"].to_numpy(dtype=float)
    occ_lon = occ["LONGITUDE"].to_numpy(dtype=float)

    # Para cada ocorrência calcula distâncias para todas as estações e escolhe a mais próxima dentro do raio
    for i in range(len(occ)):
        lat_o = occ_lat[i]
        lon_o = occ_lon[i]
        if np.isnan(lat_o) or np.isnan(lon_o):
            continue
        # calcula vetor de distâncias (km)
        dists = distancia_euclidiana_km(lat_o, lon_o, stations_lat, stations_lon)
        # encontra índices dentro do raio
        within_idx = np.where(dists <= raio_km)[0]
        if within_idx.size == 0:
            # nenhuma estação dentro do raio
            occ.at[i, "Estacao_reatribuida"] = "sem estação correspondente"
            occ.at[i, "Dist_km_reatribuida"] = np.nan
        else:
            # escolhe a estação com menor distância
            best = within_idx[np.argmin(dists[within_idx])]
            occ.at[i, "Estacao_reatribuida"] = stations_names[best]
            occ.at[i, "Dist_km_reatribuida"] = float(np.round(dists[best], 4))

    # normalizar nome da coluna para o merge com df_chuva_unificada
    occ_for_merge = occ.rename(columns={"Estacao_reatribuida": "Estacao", "DATA": "DATA_ocorrencia"})

    # filtra apenas ocorrências cuja estação atribuída existe na base de chuva
    valid_stations = set(df_chuva_unificada["Estacao"].unique())
    occ_for_merge = occ_for_merge[ occ_for_merge["Estacao"].isin(valid_stations) ].copy()

    # cria data_dia (já existe) -> garantir nome igual
    occ_for_merge["data_dia"] = pd.to_datetime(occ_for_merge["data_dia"])

    # PREPARA df_chuva_unificada para merge: garantir data_dia como date (sem horario)
    chuva = df_chuva_unificada.copy()
    chuva["data_dia"] = pd.to_datetime(chuva["data_dia"])

    # Executa merge por Estacao + data_dia (left join: mantém ocorrências)
    df_expand = pd.merge(
        occ_for_merge,
        chuva,
        on=["Estacao", "data_dia"],
        how="left",
        suffixes=("_ocorr", "_chuva")
    )

    # Estatísticas por LOCAL/DATA/Estacao (min/max/mean de Chuva_mm)
    estat = (
        df_expand
        .groupby(['LOCAL', 'DATA_ocorrencia', 'Estacao'])['Chuva_mm']
        .agg(['min','max','mean'])
        .reset_index()
    )

    # Salva arquivos de saída específicos para este raio
    nome_base = f"DADOS_GEOCODED_UNIFICADOS_{str(raio_km).replace('.',',')}km.xlsx"
    caminho_out_expand = os.path.join(out_folder_bi, nome_base)
    df_expand.to_excel(caminho_out_expand, index=False)

    nome_estat = f"ESTATISTICAS_EVENTOS_{str(raio_km).replace('.',',')}km.xlsx"
    caminho_out_estat = os.path.join(out_folder_bi, nome_estat)
    estat.to_excel(caminho_out_estat, index=False)

    # resumo: contagens
    total_ocorr = len(occ)
    atribuídas = df_expand['LOCAL'].nunique() if 'LOCAL' in df_expand.columns else len(df_expand)
    sem_estacao = total_ocorr - len(occ_for_merge)
    estações_usadas = df_expand['Estacao'].nunique()
    resumo = {
        "raio_km": raio_km,
        "total_ocorrencias_input": total_ocorr,
        "ocorrencias_com_estacao_dentro_raio": len(occ_for_merge),
        "ocorrencias_sem_estacao_dentro_raio": sem_estacao,
        "estações_distintas_usadas": estações_usadas
    }
    resumo_df = pd.DataFrame([resumo])
    resumo_df.to_csv(os.path.join(out_folder_bi, f"RESUMO_ATRIBUICAO_{str(raio_km).replace('.',',')}km.csv"), index=False)

    print(f"Raio {raio_km} km -> ocorrencias atribuídas: {len(occ_for_merge)} / {total_ocorr} | estações usadas: {estações_usadas}")
    print(f"Salvos: {os.path.basename(caminho_out_expand)} , {os.path.basename(caminho_out_estat)} , RESUMO CSV")

print("\n=== FIM DO PROCESSAMENTO PARA TODOS OS RAIOS ===")



=== Processando raio 1.5 km ===
Raio 1.5 km -> ocorrencias atribuídas: 107 / 363 | estações usadas: 13
Salvos: DADOS_GEOCODED_UNIFICADOS_1,5km.xlsx , ESTATISTICAS_EVENTOS_1,5km.xlsx , RESUMO CSV

=== Processando raio 5 km ===
Raio 5 km -> ocorrencias atribuídas: 360 / 363 | estações usadas: 16
Salvos: DADOS_GEOCODED_UNIFICADOS_5km.xlsx , ESTATISTICAS_EVENTOS_5km.xlsx , RESUMO CSV

=== Processando raio 10 km ===
Raio 10 km -> ocorrencias atribuídas: 363 / 363 | estações usadas: 16
Salvos: DADOS_GEOCODED_UNIFICADOS_10km.xlsx , ESTATISTICAS_EVENTOS_10km.xlsx , RESUMO CSV

=== Processando raio 20 km ===
Raio 20 km -> ocorrencias atribuídas: 363 / 363 | estações usadas: 16
Salvos: DADOS_GEOCODED_UNIFICADOS_20km.xlsx , ESTATISTICAS_EVENTOS_20km.xlsx , RESUMO CSV

=== FIM DO PROCESSAMENTO PARA TODOS OS RAIOS ===
